<a href="https://colab.research.google.com/github/rashida048/ApacheSpark-BigDataAnalytics/blob/main/feature_select_ml_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 64 kB/s 
     |████████████████████████████████| 198 kB 38.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=ad9942c0b6783219b052df3bc9ed34d50d3ca2d37569af6fa18b29e9ca6fe858
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [3]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
import re
import numpy as np

In [5]:
def buildArray(listOfIndices):
  returnVal = np.zeros(20000)

  for index in listOfIndices:
    returnVal[index] = returnVal[index] + 1

  mysum = np.sum(returnVal)

  returnVal = np.divide(returnVal, mysum)
  return returnVal

In [6]:
def changeBinary(x):
  if "AU" in x:
    return 1
  else:
    return 0

In [7]:
def conversion(rd):
  validLines = rd.filter(lambda x: 'id' in x and 'url=' in x)
  keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
  regex = re.compile('[^a-zA-Z]')
  keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))
  allWords = keyAndListOfWords.flatMap(lambda x: [(y, 1) for y in x[1]])
  allCounts = allWords.reduceByKey(lambda x, y: x+y)
  topWords = allCounts.top(20000, lambda x: x[1])
  topWordsK = sc.parallelize(range(20000))
  dictionary = topWordsK.map (lambda x : (topWords[x][0], x))
  allWordsWithDocID = keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))
  allDictionaryWords = allWordsWithDocID.join(dictionary)
  justDocAndPos = allDictionaryWords.map(lambda x: x[1])
  allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()
  allDocsAsNumpyArrays = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(list(x[1]))))
  xy = allDocsAsNumpyArrays.map(lambda x: (changeBinary(x[0]), x[1]))
  xy1 = xy.map(lambda x: (x[0], x[1].tolist()))
  xy_df = xy1.toDF(["lab", "feat"])
  return (xy, xy_df)

In [8]:
import timeit
task3_total_strt = timeit.default_timer()
strt_total = timeit.default_timer()

strt_read = timeit.default_timer()
rd = sc.textFile("SmallTrainingData.txt")
stp_read = timeit.default_timer()
print((stp_read - strt_read)/60, "minutes to read the training data")

0.017554699366667138 minutes to read the training data


In [9]:
conv = conversion(rd)

In [10]:
rdd = conv[0]

In [11]:
import random
ran = random.sample(range(0, 20000), 10000)

In [12]:
rdd.take(3)

[(1, array([0.11883862, 0.05536799, 0.00675219, ..., 0.        , 0.        ,
         0.        ])),
 (1, array([0.10305958, 0.04508857, 0.01046699, ..., 0.        , 0.        ,
         0.        ])),
 (1, array([0.08316993, 0.04428105, 0.02320261, ..., 0.        , 0.        ,
         0.        ]))]

In [21]:
reduced_rdd = rdd.map(lambda x: (x[0], (x[1][ran]).tolist()))

In [22]:
df = reduced_rdd.toDF(['lab', 'feat'])

In [23]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

In [24]:
convers = udf(lambda x: Vectors.dense(x), VectorUDT())
df1 = df.withColumn("features", convers("feat")).drop("feat")

In [25]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LinearSVC

In [27]:
lr_train_strt = timeit.default_timer()
lr_red = LogisticRegression(featuresCol="features", labelCol='lab',
                        maxIter = 20)
lrmodel_new = lr_red.fit(df1)

lr_train_stp = timeit.default_timer()
print((lr_train_stp-lr_train_strt)/60, "minutes to train the logistic regression with reduced features")

1.873271036383335 minutes to train the logistic regression with reduced features


In [28]:
strt_read_test = timeit.default_timer()
rd1 = sc.textFile("SmallTrainingData.txt")
stp_read_test = timeit.default_timer()
print((stp_read_test - strt_read_test)/60, "minutes to read the testing data")

0.0011821181666694732 minutes to read the testing data


In [29]:
conv1 = conversion(rd1)

In [32]:
rdd1 = conv1[0]

In [33]:
reduced_rdd_test = rdd1.map(lambda x: (x[0], (x[1][ran]).tolist()))

In [34]:
df_test = reduced_rdd_test.toDF(['lab', 'feat'])

In [35]:
df1_test = df_test.withColumn("features", convers("feat")).drop("feat")

In [36]:
lr_test_strt = timeit.default_timer()
lr_selected_pred = lrmodel_new.transform(df1_test)

multi_eval = MulticlassClassificationEvaluator(labelCol="lab", metricName = "accuracy")
print('Logistic Regression Accuracy on Selected Features:', multi_eval.evaluate(lr_selected_pred))

multi_eval1 = MulticlassClassificationEvaluator(labelCol="lab", metricName = "f1")
print('Logistic Regression F1 on Selected Features:', multi_eval1.evaluate(lr_selected_pred))

lr_test_stp = timeit.default_timer()

lr_test_reduced = (lr_test_stp - lr_test_strt)/60
print(lr_test_reduced, "minutes to test the logistic regression with reduced features")

Logistic Regression Accuracy on Selected Features: 1.0
Logistic Regression F1 on Selected Features: 1.0
1.8161702978833318 minutes to test the logistic regression with reduced features


In [37]:
task3_total_stp = timeit.default_timer()
total_logistic = (task3_total_stp - task3_total_strt)/60
print(total_logistic, "minutes total to complete the logistic regression")

17.007684969416665 minutes total to complete the logistic regression


In [39]:
svc_train_stp = timeit.default_timer()

lsvc = LinearSVC(featuresCol="features", labelCol='lab', maxIter=10, regParam=0.1)
lsvcModel = lsvc.fit(df1_test)

svc_train_strt = timeit.default_timer()

svc_train = (svc_train_strt - svc_train_stp)/60
print(svc_train, "minutes to train the support vector with reduced features")

1.9175520145499982 minutes to train the support vector with reduced features


In [40]:
svc_test_stp = timeit.default_timer()

svc_select_pred = lsvcModel.transform(df1_test)

multi_eval1 = MulticlassClassificationEvaluator(labelCol="lab", metricName = "accuracy")
print('SVC Accuracy on selected features:', multi_eval1.evaluate(svc_select_pred))

multi_eval_f1 = MulticlassClassificationEvaluator(labelCol="lab", metricName = "f1")
print('SVC F1 on selected features:', multi_eval_f1.evaluate(svc_select_pred))

svc_test_strt = timeit.default_timer()
svc_test = (svc_test_strt - svc_test_stp)/60
print(svc_test, "minutes to test the support vector with reduced features")

SVC Accuracy on selected features: 1.0
SVC F1 on selected features: 1.0
1.8208114590833324 minutes to test the support vector with reduced features


In [41]:
end_time = timeit.default_timer()
total_time = (end_time - task3_total_strt)/60

print(total_time - (lr_train_stp-lr_train_strt)/60 - lr_test_reduced, "minutes total to complete SVC with reduced feature")

20.508096048266665 minutes total to complete SVC with reduced feature
